In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix

C:\Users\Name\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
bank =pd.read_csv('bank-full.csv',sep=';')

In [5]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [46]:
bank =pd.read_csv('bank-full.csv',sep=';')
bank_temp=bank.drop('y',axis=1)
bank_dummies=pd.get_dummies(bank_temp)
bank_dummies['y']=bank['y']
train,test =train_test_split(bank_dummies,test_size=0.3,random_state=100)
train_y =train['y']
test_y = test['y']
train_x=train.drop('y',axis=1)
test_x=test.drop('y',axis=1)
model = DecisionTreeClassifier(random_state=100)
model.fit(train_x,train_y)
test_pred = model.predict(test_x)
df_pred = pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred['pred_status']=df_pred['actual'] ==df_pred['predicted']
dt_accuracy=df_pred[df_pred['pred_status']==True].shape[0] / df_pred.shape[0] *100
dt_accuracy

87.72485992332645

In [11]:
tp =df_pred[(df_pred['predicted']=='yes') &(df_pred['actual']=='yes')].shape[0]
tn =df_pred[(df_pred['predicted']=='no') &(df_pred['actual']=='no')].shape[0]
fp =df_pred[(df_pred['predicted']=='yes') &(df_pred['actual']=='no')].shape[0]
fn=df_pred[(df_pred['predicted']=='no') &(df_pred['actual']=='yes')].shape[0]
print("true positive:%d" %tp)
print("true negative:%d" %tn)
print("false positive:%d" %fp)
print("false negative:%d" %fn)

true positive:775
true negative:11124
false positive:861
false negative:804


In [17]:
dt_sensitivity = (tp/(tp+fn)) *100
dt_sensitivity

49.08169727675744

In [18]:
dt_specificity = (tn/(tn+fp)) *100
dt_specificity

92.81602002503129

In [22]:
model_rf = RandomForestClassifier(random_state=100)
model_rf.fit(train_x,train_y)
test_pred = model_rf.predict(test_x)
df_pred_rf= pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred_rf['pred_status']=df_pred_rf['actual'] ==df_pred_rf['predicted']
rf_accuracy =df_pred_rf[df_pred_rf['pred_status']==True].shape[0] / df_pred_rf.shape[0] *100
rf_accuracy

89.97345915659098

In [23]:
tp_rf =df_pred_rf[(df_pred_rf['predicted']=='yes') &(df_pred_rf['actual']=='yes')].shape[0]
tn_rf =df_pred_rf[(df_pred_rf['predicted']=='no') &(df_pred_rf['actual']=='no')].shape[0]
fp_rf =df_pred_rf[(df_pred_rf['predicted']=='yes') &(df_pred_rf['actual']=='no')].shape[0]
fn_rf=df_pred_rf[(df_pred_rf['predicted']=='no') &(df_pred_rf['actual']=='yes')].shape[0]
print("true positive:%d" %tp_rf)
print("true negative:%d" %tn_rf)
print("false positive:%d" %fp_rf)
print("false negative:%d" %fn_rf)

true positive:561
true negative:11643
false positive:342
false negative:1018


In [24]:
rf_sensitivity = (tp_rf/(tp_rf+fn_rf)) *100
rf_sensitivity

35.52881570614313

In [25]:
rf_specificity = (tn_rf/(tn_rf+fp_rf)) *100
rf_specificity

97.14643304130163

In [38]:
model_ab = AdaBoostClassifier(random_state=100)
model_ab.fit(train_x,train_y)

test_pred =model_ab.predict(test_x)
df_pred_ab= pd.DataFrame({'actual':test_y,'predicted':test_pred})
df_pred_ab['pred_status']=df_pred_ab['actual'] ==df_pred_ab['predicted']
ab_accuracy =df_pred_ab[df_pred_ab['pred_status']==True].shape[0] / df_pred_ab.shape[0] *100
ab_accuracy


89.72279563550575

In [39]:
tn_ab,fp_ab,fn_ab,tp_ab = confusion_matrix(df_pred_ab['actual'],df_pred_ab['predicted']).ravel()
tn_ab,fp_ab,fn_ab,tp_ab

(11585, 400, 994, 585)

In [40]:
ab_sensitivity =tp_ab/(tp_ab+fn_ab)  * 100

In [41]:
ab_specificity =tn_ab/(tn_ab+fp_ab) *100

In [42]:
model_dict={"Decision_Tree":pd.Series([dt_accuracy,dt_sensitivity,dt_specificity,tp,tn,fp,fn]),
            "Random_Forest":pd.Series([rf_accuracy,rf_sensitivity,rf_specificity,tp_rf,tn_rf,fp_rf,fn_rf]),
             "Adaptive Boosting":pd.Series([ab_accuracy,ab_sensitivity,ab_specificity,tp_ab,tn_ab,fp_ab,fn_ab])}
model_df = pd.DataFrame(model_dict)
model_df.index=["Accuracy","Sensitivity","Specificity","tp","fn","fp","fn"]
print(model_df)

             Adaptive Boosting  Decision_Tree  Random_Forest
Accuracy             89.722796      87.724860      89.973459
Sensitivity          37.048765      49.081697      35.528816
Specificity          96.662495      92.816020      97.146433
tp                  585.000000     775.000000     561.000000
fn                11585.000000   11124.000000   11643.000000
fp                  400.000000     861.000000     342.000000
fn                  994.000000     804.000000    1018.000000


# machine learning steps
- read the data
- check the output column datatype
  -  if the output data type is integer dont do anything,if it is object drop it temporarily
- send the temporary data to dummies
- it will convert it into all numeric columns
- now concat the dropped column
- now split the data into train and test 
- now send all the input columns into trian_x,test_x
- all output columns into train_y,test_y
- now build the model
- send both train_x and train_y to it i.e both input and output
- to predict values send test input data i.e test_x
- after getting the values compare with the actual values and create a data frame
- calculate the accuracy i.e tp+tn/(tp_tn+fp+fn) or number of true predictions/number of total values
- using confusion matrix find tp,tn,fp,fn
- calculate both specificity,sensitivity


# Gini Impurity steps
- check the overall gini impurity by dividing the values in output columns
- check: 1- np.square(p(1))- np.square(p(0)) i.e p(1)= number of 1s/number of total rows
- now see for which column we get the gini impurity less by using that column as first decision tree
- if we use gender_female column then calculate the gini_right and gini_left and combine the gini overall see 
  if it is less than giniwhole
- like that check for all columns see which column gives gini impurity less that is the column used by model for classification